##### Imports

In [1]:
import sys
from pathlib import Path
import warnings

import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import sys
# Custom library paths
sys.path.extend(['../', './scr'])

from scr.utils import set_seed
from scr.utils import read_words
from pathlib import Path
import random
from collections import Counter, defaultdict
import pickle
from tqdm import tqdm
from torch.utils.data import Dataset

from scr.utils import read_words, save_words_to_file

import pickle
from pathlib import Path
from scr.dataset import *
from scr.utils import *
# # For inference
from scr.feature_engineering import *

import gc

set_seed(42)

import torch
import torch.nn as nn
from pathlib import Path
import random

from scr.utils import print_scenarios
torch.set_float32_matmul_precision('medium')
from pathlib import Path

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Read and Shuffle Word List
word_list = read_words('data/words_250000_train.txt') # , limit=10000)
# word_list = read_words('data/250k.txt', limit=10000)

random.shuffle(word_list)

# Calculate Frequencies and Max Word Length
word_frequencies = calculate_word_frequencies(word_list)
char_frequency = calculate_char_frequencies(word_list)
max_word_length = max(len(word) for word in word_list)

##### Data Dir

In [2]:
NUM_STRATIFIED_SAMPLES = 2_00 # This will be overwritten by Papermill

NUM_WORD_SAMPLE = 1_000 # words for testing

FAST_DEV_RUN = False

MAX_EPOCH = 250

In [3]:
from pathlib import Path
from scr.custom_sampler import *

# Define the base directory and the paths for training and validation parquet files
base_dataset_dir = Path("/media/sayem/510B93E12554BBD1/dataset/")
stratified_samples_dir = base_dataset_dir / str(NUM_STRATIFIED_SAMPLES)
parquet_path = stratified_samples_dir / 'parquets'

# Create directories for train and validation parquets if they don't exist
parquet_path.mkdir(parents=True, exist_ok=True)
# parquet_valid_path.mkdir(parents=True, exist_ok=True)

# Define and create the directory for models
models_dir = Path("/home/sayem/Desktop/Hangman/models")
models_dir.mkdir(parents=True, exist_ok=True)

# Define your output directory
# Define your output directory and logger directory
output_dir = Path("/home/sayem/Desktop/Hangman/training_outputs")
logger_dir = output_dir / "lightning_logs"

# Create the output and logger directories if they don't exist
output_dir.mkdir(parents=True, exist_ok=True)
logger_dir.mkdir(parents=True, exist_ok=True)

# Define the file path for saving the testing words
testing_words_file_path = stratified_samples_dir / "testing_words.txt"

try:
    testing_word_list = read_words(testing_words_file_path)
    print(f"Length of the testing word list: {len(testing_word_list)}")
    sampled_test_words = stratified_sample_by_length_and_uniqueness(testing_word_list, NUM_WORD_SAMPLE)
    print(f"Sampled {len(sampled_test_words)} unique words for testing.")
except FileNotFoundError:
    print(f"File not found: {testing_words_file_path}")

print(len(sampled_test_words))

Length of the testing word list: 10048
Sampled 1085 unique words for testing.
1085


##### Dataset Loading and train test split

In [4]:
# Create datasets directly from the saved parquet files
hangman_dataset = HangmanDataset(parquet_path)
# valid_dataset = HangmanDataset(parquet_valid_path)

from scr.utils import *

# print(len(train_dataset))
# print(len(valid_dataset))

# assert len(train_dataset) > len(valid_dataset)

# Assuming `hangman_dataset` is an instance of HangmanDataset
train_dataset, valid_dataset = split_hangman_dataset(hangman_dataset)

In [5]:
hangman_dataset[(10,)]

{'game_id': 2884,
 'word': 'antanagoge',
 'initial_state': ['_n__n_goge'],
 'final_state': '_nt_n_goge',
 'guessed_states': ['_n__n_goge',
  '_nt_n_goge',
  '_nt_n_goge',
  '_nt_n_goge',
  '_nt_n_goge',
  '_nt_n_goge',
  '_nt_n_goge'],
 'guessed_letters': ['t', 'z', 's', 'h', 'q', 'f', 'y'],
 'game_state': 'midLateRevealed',
 'difficulty': 'hard',
 'outcome': 'win',
 'word_length': 10,
 'won': False}

In [6]:
# Sample data
sample_game = {
    'guessed_states': ['_d__d__e__', '_d__d__e__', '_d__d__e__', '_d__d_me__', \
        '_d__d_me__', '_d__d_me__', '_d__d_men_', '_d__d_men_'],
    'guessed_letters': ['c', 'r', 'm', 'v', 'i', 'n', 'w', 'o']
}

In [7]:
from scr.feature_engineering import *

In [8]:
# Define parameters
max_word_length = 29  # Set as per your model's requirement
max_seq_length = 10  # Set as per your model's requirement

# # Dummy character frequency (example, create as per your data)
# char_frequency = {char: 1 for char in 'abcdefghijklmnopqrstuvwxyz'}

# Call the function and unpack the returned tuple into two variables
char_sequence_tensor, missed_chars_tensor = process_game_sequence(
    game_states=sample_game['guessed_states'], 
    guessed_letters_sequence=sample_game['guessed_letters'],
    char_frequency=char_frequency, 
    max_word_length=max_word_length,
    max_seq_length=max_seq_length
)

# Now you can print the shape of each tensor separately
print(f"Character Sequence Tensor Shape: {char_sequence_tensor.shape}")
print(f"Missed Chars Tensor Shape: {missed_chars_tensor.shape}")

Character Sequence Tensor Shape: torch.Size([8, 154])
Missed Chars Tensor Shape: torch.Size([8, 28])


In [9]:
from scr.feature_engineering import *

In [10]:
# Sample data for a batch of games
batch_sample_games = {
    'guessed_states_batch': [
        ['_d__d__e__'],
        # ['_b__b__t__'],
        # ['__ll___p__'],
    ],
    
    'guessed_letters_batch': [
        ['c'],
        # ['a'],
        # ['i']
    ]
}

In [11]:
# # Sample data for a batch of games
# batch_sample_games = {
#     'guessed_states_batch': [
#         ['_d__d__e__', '_d__d__e__', '_d__d__e__', '_d__d_me__', '_d__d_me__', '_d__d_me__', '_d__d_men_', '_d__d_men_'],
#         ['_b__b__t__', '_b__b__t__', '_b__b__t__', '_b__b_t__', '_b__b_te__', '_b__b_te__', '_b__b_test', '_b__b_test'],
#         ['__ll___p__', '__ll___p__', '__ll___p__', '__ll___p__', '__ll___p__', '__ll__op_', '__ll__op_', '__ll__op_']
#     ],
    
#     'guessed_letters_batch': [
#         ['c', 'r', 'm', 'v', 'i', 'n', 'w', 'o'],
#         ['a', 's', 'e', 'f', 't', 'r', 'n', 'u'],
#         ['i', 'q', 'r', 't', 'p', 'o', 'a', 's']
#     ]
# }

# Define parameters
max_word_length = 29  # Set as per your model's requirement
max_seq_length = 10  # Set as per your model's requirement

# # Dummy character frequency (example, create as per your data)
# char_frequency = {char: 1 for char in 'abcdefghijklmnopqrstuvwxyz'}

# Call the function
batch_features, batch_missed_chars = process_batch_of_games(
    batch_sample_games['guessed_states_batch'],
    batch_sample_games['guessed_letters_batch'],
    char_frequency,
    max_word_length,
    max_seq_length=1
)

print()
# Print shapes for sanity check
print(f"Batch Features Shape: {batch_features.shape}")
print(f"Batch Missed Chars Shape: {batch_missed_chars.shape}")


Batch Features Shape: torch.Size([1, 1, 154])
Batch Missed Chars Shape: torch.Size([1, 1, 28])


In [12]:
batch_sample_games['guessed_states_batch'][0]

['_d__d__e__']

In [13]:
batch_sample_games['guessed_letters_batch'][0]

['c']

In [14]:
# Sample data
sample_game = {
    'guessed_states': ['_d__d__e__', '_d__d__e__', '_d__d__e__', '_d__d_me__', \
        '_d__d_me__', '_d__d_me__', '_d__d_men_', '_d__d_men_'],
    'guessed_letters': ['c', 'r', 'm', 'v', 'i', 'n', 'w', 'o']
}

In [15]:
overall_sucess_rate, guess_outcome = analyze_guess_outcomes(sample_game['guessed_states'], \
    sample_game['guessed_letters'], maximum_word_length=29)

In [16]:
overall_sucess_rate

0.07142857142857142

In [17]:
guess_outcome

[0, 0, 1, 0, 0, 1, 0]

In [18]:
game_states = ['allMasked', 'early', 'quarterRevealed', 'midRevealed', 
               'midLateRevealed', 'lateRevealed', 'nearEnd']
game_state_to_idx = {state: idx for idx, state in enumerate(game_states)}

def encode_game_state(game_state):
    state_vector = [0] * len(game_states)
    state_index = game_state_to_idx.get(game_state, -1)
    if state_index >= 0:
        state_vector[state_index] = 1
    return state_vector

In [19]:
from scr.data_module import *

# Initialize Data Module
initial_batch_size = 1  # Set your initial batch size

# Initialize Data Module with the required arguments
data_module = HangmanDataModule(train_dataset, valid_dataset, 
                                initial_batch_size, 
                                custom_collate_fn)
                                # performance_metrics=None)

# for batch in data_module.train_dataloader():
#     # print(batch)

# batch_features, batch_missed_chars = process_batch(next(iter(data_module.train_dataloader())), \
#     char_frequency, max_word_length)

# print(batch_features.shape)
# print(batch_missed_chars.shape)

# # break

for batch in data_module.train_dataloader():
    states = batch['guessed_states']
    # print(states.shape)
    guesses = batch['guessed_letters']
    max_seq_length = batch['max_seq_len']
    original_seq_lengths = batch['original_seq_lengths']

    # Print debug information
    print("DEBUG INFO:")
    # Uncomment these if needed
    # print(f"Miss Penalty: {miss_penalty}")
    # print(f"Type of Miss Penalty: {type(miss_penalty)}")
    print(f"Batch Word Lengths: {batch['word_length']}")
    print(f"Batch Difficulties: {batch['difficulty']}")
    print(f"Batch Outcomes: {batch['outcome']}")
    print(f"Batch Won Flags: {batch['won']}")

    # Print the entire batch
    print("Batch Contents:")
    for key, value in batch.items():
        print(f"{key}: {value}")

    batch_features, batch_missed_chars = process_batch_of_games(
        states, char_frequency,
        max_word_length,
        max_seq_length)

    print()
    print(f"{batch_features.shape}")
    print(f"{batch_missed_chars.shape}")
    break

Normal sampler kicked...
DEBUG INFO:
Batch Word Lengths: [20]
Batch Difficulties: ['easy']
Batch Outcomes: ['win']
Batch Won Flags: [False]
Batch Contents:
guessed_states: [['noncon_escen_ingness']]
guessed_letters: [['d']]
max_seq_len: 1
original_seq_lengths: [1]
difficulty: ['easy']
outcome: ['win']
word_length: [20]
won: [False]


TypeError: process_batch_of_games() missing 1 required positional argument: 'max_seq_length'

In [ ]:
miss_char.shape

In [ ]:
# # # # Validate the model (if needed)
# trainer.validate(model=lightning_model, datamodule=data_module)
print(f"Training Begin for {NUM_STRATIFIED_SAMPLES} words: {len(train_dataset)} Games")
# # # # # # Fit the model
trainer.fit(lightning_model, data_module)

# # Optionally print the profiler summary
# print(profiler.summary())

# Save the entire modell
trained_model_file = models_dir / f"{NUM_STRATIFIED_SAMPLES}_trained_model.pth"
torch.save(lightning_model, trained_model_file)
print(f"Model saved at {trained_model_file}")

In [ ]:
batch_features.shape

In [ ]:
batch_missed_chars.shape

In [ ]:
features, missed_chars = build_enhanced_feature_set(sample, \
    char_frequency, max_word_length)

In [ ]:
features.shape

In [ ]:
STOP

In [ ]:
import torch
import torch.nn as nn

# Sample dimensions
missed_char_dim = 28  # Dimension of missed character vector
hidden_dim = 10       # Arbitrary hidden dimension for output

class TestModel(nn.Module):
    def __init__(self):
        super(TestModel, self).__init__()
        self.miss_linear = nn.Linear(missed_char_dim, hidden_dim)

    def forward(self, missed_chars):
        missed_chars_processed = self.miss_linear(missed_chars)
        return missed_chars_processed

# Create model
model = TestModel()

# Sample data: a batch of size 1 with 28 missed character indicators
# Creating a sample input with 28 values, each being 0 or 1
missed_chars = torch.tensor([0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, \
    1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1], dtype=torch.float)

# Forward pass
output = model(missed_chars)
output.shape

In [ ]:
def calculate_difficulty_score(metrics):
    # Extracting the metrics
    win_rate = metrics.get('performance_wins', 0)
    avg_attempts = metrics.get('performance_total_attempts_used', 0)
    miss_penalty = metrics.get('miss_penalty_avg', 0)

    # Weights for each metric (these can be adjusted)
    weight_win_rate = 1.0   # Higher weight as win rate is a strong indicator of difficulty
    weight_avg_attempts = 0.5  # Moderate weight
    weight_miss_penalty = 0.5  # Moderate weight

    # Normalize the metrics (invert win rate as lower win rate indicates higher difficulty)
    normalized_win_rate = (100 - win_rate) / 100
    # normalized_avg_attempts = avg_attempts / 6  # Assuming max avg_attempts is 6
    normalized_miss_penalty = miss_penalty  # Already in range 0 to 1

    # Calculate the composite score
    composite_score = (
        weight_win_rate * normalized_win_rate +
        weight_miss_penalty * normalized_miss_penalty
    )

    return composite_score

# Example usage
metrics = {
    'performance_wins': 0,  # Example values
    # 'performance_total_attempts_used': 6,
    'miss_penalty_avg': 1
}

score = calculate_difficulty_score(metrics)
print("Difficulty Score:", score)

# # Use the score to determine if the word length should be added to target pairs
# if score >= 0.001:  # Define this threshold based on your game's difficulty scale
#     target_pairs.append((int(word_length),))


In [ ]:
def calculate_difficulty_score(metrics):
    # Extracting the metrics
    win_rate = metrics.get('performance_wins', 0)
    miss_penalty = metrics.get('miss_penalty_avg', 0)

    # Weights for each metric
    weight_win_rate = 1.0   # Higher weight for win rate
    weight_miss_penalty = 0.5  # Weight for miss penalty

    # Normalize the metrics (invert win rate as lower win rate indicates higher difficulty)
    normalized_win_rate = (100 - win_rate) / 100
    normalized_miss_penalty = miss_penalty  # Already in range 0 to 1

    # Calculate the composite score
    composite_score = (
        weight_win_rate * normalized_win_rate +
        weight_miss_penalty * normalized_miss_penalty
    )

    return composite_score


# Best-case scenario metrics
best_case_metrics = {
    'performance_wins': 100,  # Maximum win rate
    'miss_penalty_avg': 0     # Minimum miss penalty
}

best_case_score = calculate_difficulty_score(best_case_metrics)
print("Best-Case Difficulty Score:", best_case_score)

In [ ]:
# Worst-case scenario metrics
worst_case_metrics = {
    'performance_wins': 0,  # Minimum win rate
    'miss_penalty_avg': 1   # Maximum miss penalty
}

worst_case_score = calculate_difficulty_score(worst_case_metrics)
print("Worst-Case Difficulty Score:", worst_case_score)

In [ ]:
def calculate_difficulty_score(metrics, weight_win_rate=1.0, weight_miss_penalty=0.5):
    """
    Calculates the difficulty score based on win rate and miss penalty.
    
    :param metrics: Dictionary containing 'performance_wins' and 'miss_penalty_avg'.
    :param weight_win_rate: Weight for the win rate metric.
    :param weight_miss_penalty: Weight for the miss penalty metric.
    :return: Calculated difficulty score.
    """
    # Extracting the metrics
    win_rate = metrics.get('performance_wins', 0)
    miss_penalty = metrics.get('miss_penalty_avg', 0)

    # Normalize the metrics (invert win rate as lower win rate indicates higher difficulty)
    normalized_win_rate = (100 - win_rate) / 100
    normalized_miss_penalty = miss_penalty  # Already in range 0 to 1

    # Calculate the composite score
    composite_score = (
        weight_win_rate * normalized_win_rate +
        weight_miss_penalty * normalized_miss_penalty
    )

    return composite_score

# Example usage with custom weights
metrics = {
    'performance_wins': 0,  # Example values
    'miss_penalty_avg': 1
}

# Custom weights
custom_weight_win_rate = 1.0
custom_weight_miss_penalty = 0.5

score = calculate_difficulty_score(metrics, custom_weight_win_rate, custom_weight_miss_penalty)
print("Difficulty Score with Custom Weights:", score)

In [ ]:
aggregated_metrics = {
    5: {'total_games': 100, 'wins': 60, 'total_attempts_used': 300, 'win_rate': 0.6, \
        'average_attempts_used': 3.0, 'miss_penalty': 0.02},
    6: {'total_games': 150, 'wins': 90, 'total_attempts_used': 450, 'win_rate': 0.6, \
        'average_attempts_used': 3.0, 'miss_penalty': 0.03}
}

# Iterating over the dictionary
for word_len, metrics in aggregated_metrics.items():
    print(f"Word Length: {word_len}")
    for key, value in metrics.items():
        print(f"  {key}: {value}")

In [ ]:
def select_target_pairs(performance_metrics, batch_size):
    target_pairs = []

    for word_length, metrics in performance_metrics.items():
        # print(metrics)
        win_rate = metrics.get('win_rate', 0)
        avg_attempts = metrics.get('average_attempts_used', 0)

        if win_rate <= 20 and avg_attempts >= 4:
            target_pairs.append(word_length)

    return target_pairs[:batch_size]

# Test data
performance_metrics = {
    5: {'total_games': 100, 'wins': 60, 'total_attempts_used': 300, 'win_rate': 0.6, 
        'average_attempts_used': 4.0, 'miss_penalty': 0.02},
    6: {'total_games': 150, 'wins': 90, 'total_attempts_used': 450, 'win_rate': 0.6, 
        'average_attempts_used': 4.0, 'miss_penalty': 0.03}
}

# Test the function
batch_size = 10
target_pairs = select_target_pairs(performance_metrics, batch_size)
print("Target Pairs:", target_pairs)

In [ ]:
# def calculate_miss_penalty(outputs, miss_chars):
#     if outputs.numel() == 0:
#         print("Empty outputs tensor")
#         return torch.tensor(0.0, device=outputs.device)

#     miss_penalty = torch.sum(outputs * miss_chars) / outputs.numel()
#     return miss_penalty

In [ ]:
from scr.feature_engineering import *

In [ ]:
word = 'cat'

encoded = encode_word(word)

encoded

In [ ]:
# Get missed characters tensor
miss_chars = get_missed_characters(word)

In [ ]:
miss_chars

In [ ]:
# Creating a dummy output tensor for the word 'cat'
word = 'cat'

# Assumptions for the dummy output
batch_size = 1  # Number of words in the batch
max_seq_len = len(word)  # Maximum sequence length (length of the word)
num_characters = 28  # Total number of characters in the vocabulary

# Creating the dummy output tensor
# For simplicity, filling it with random values between 0 and 1
outputs = torch.rand((batch_size, max_seq_len, num_characters))

outputs.shape, outputs

In [ ]:
# probabilities = F.softmax(outputs, dim=-1)

# probabilities

In [ ]:
# Creating two dummy output tensors for the word 'cat'
# One where all characters are accurately predicted and another where all are wrong

# Accurate Predictions: Setting the confidence for the correct characters ('c', 'a', 't') to 1
# and others to 0 for each position in the word
correct_outputs = torch.zeros((batch_size, max_seq_len, num_characters))
correct_outputs[0, 0, char_to_idx['c']] = 1  # High confidence for 'c' in the first position
correct_outputs[0, 1, char_to_idx['a']] = 1  # High confidence for 'a' in the second position
correct_outputs[0, 2, char_to_idx['t']] = 1  # High confidence for 't' in the third position

# Wrong Predictions: Setting the confidence for incorrect characters to 1 and for correct ones to 0
wrong_outputs = torch.ones((batch_size, max_seq_len, num_characters))
wrong_outputs[0, 0, char_to_idx['c']] = 0  # Zero confidence for 'c' in the first position
wrong_outputs[0, 1, char_to_idx['a']] = 0  # Zero confidence for 'a' in the second position
wrong_outputs[0, 2, char_to_idx['t']] = 0  # Zero confidence for 't' in the third position

correct_outputs.shape, wrong_outputs.shape

In [ ]:
# Calculate miss penalty
miss_penalty = calculate_miss_penalty(wrong_outputs, miss_chars)
miss_penalty

##### Data Dir

In [ ]:
NUM_STRATIFIED_SAMPLES = 100 # This will be overwritten by Papermill

NUM_WORD_SAMPLE = 1_000 # words for testing

FAST_DEV_RUN = False

MAX_EPOCH = 15

In [ ]:
from pathlib import Path
from scr.custom_sampler import *

# Define the base directory and the paths for training and validation parquet files
base_dataset_dir = Path("/media/sayem/510B93E12554BBD1/dataset/")
stratified_samples_dir = base_dataset_dir / str(NUM_STRATIFIED_SAMPLES)
parquet_path = stratified_samples_dir / 'parquets'

# Create directories for train and validation parquets if they don't exist
parquet_path.mkdir(parents=True, exist_ok=True)
# parquet_valid_path.mkdir(parents=True, exist_ok=True)

# Define and create the directory for models
models_dir = Path("/home/sayem/Desktop/Hangman/models")
models_dir.mkdir(parents=True, exist_ok=True)

# Define your output directory
# Define your output directory and logger directory
output_dir = Path("/home/sayem/Desktop/Hangman/training_outputs")
logger_dir = output_dir / "lightning_logs"

# Create the output and logger directories if they don't exist
output_dir.mkdir(parents=True, exist_ok=True)
logger_dir.mkdir(parents=True, exist_ok=True)

# Define the file path for saving the testing words
testing_words_file_path = stratified_samples_dir / "testing_words.txt"

try:
    testing_word_list = read_words(testing_words_file_path)
    print(f"Length of the testing word list: {len(testing_word_list)}")
    sampled_test_words = stratified_sample_by_length_and_uniqueness(testing_word_list, \
        NUM_WORD_SAMPLE)
    print(f"Sampled {len(sampled_test_words)} unique words for testing.")
except FileNotFoundError:
    print(f"File not found: {testing_words_file_path}")

print(len(sampled_test_words))

##### Dataset Loading

In [ ]:
# Create datasets directly from the saved parquet files
hangman_dataset = HangmanDataset(parquet_path)
# valid_dataset = HangmanDataset(parquet_valid_path)

from scr.utils import *

# Assuming `hangman_dataset` is an instance of HangmanDataset
# Usage
train_dataset, valid_dataset \
    = split_hangman_dataset(hangman_dataset, 0.8)

print(len(train_dataset))
print(len(valid_dataset))

assert len(train_dataset) > len(valid_dataset)

In [ ]:
hangman_dataset[(29,)]

In [ ]:
train_dataset[(29,)]

In [ ]:
train_dataset.get_all_group_labels()

In [ ]:
len(train_dataset)

In [ ]:
# Request a sample with word length 29
word_length = 5
sample = hangman_dataset[(word_length,)]
sample

In [ ]:
sample

In [ ]:
from scr.data_module import *
from scr.dataset import *

# Initialize Data Module
initial_batch_size = 128  # Set your initial batch size

# Initialize Data Module with the required arguments
data_module = HangmanDataModule(train_dataset, valid_dataset, 
                                initial_batch_size, 
                                custom_collate_fn)

In [ ]:
train_loader = data_module.train_dataloader()

In [ ]:
dummy_performance_metrics = {
    
    3: {'win_rate': 45, 'average_attempts_used': 5},
    4: {'win_rate': 60, 'average_attempts_used': 3},  # This won't be selected due to high win rate
    5: {'win_rate': 30, 'average_attempts_used': 6},
    6: {'win_rate': 48, 'average_attempts_used': 4},
    # ... add more dummy metrics as needed ...
}

In [ ]:
# Initialize PerformanceBasedSampler
sampler = PerformanceBasedSampler(dataset=hangman_dataset, 
                                  performance_metrics=dummy_performance_metrics, 
                                  batch_size=10)

# Verify target pairs
print("Target pairs:", sampler.target_pairs)

In [ ]:
next(iter(sampler))

In [ ]:
train_dataset[(1,)]